In [1]:
import numpy as np
import os
import copy
import itertools
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import random
import argparse
from pathlib import Path
from copy import deepcopy
import torchvision.models as models
import torchvision


from torch.utils.tensorboard import SummaryWriter

In [24]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
!pwd

/content


In [4]:
!ls

drive  sample_data


In [28]:
%cd "/content/drive/MyDrive/Courses/Sem_2_Spring_22/Independent_study"

[Errno 2] No such file or directory: '/content/drive/MyDrive/Courses/Sem_2_Spring_22/Independent_study'
/content/drive/MyDrive/Courses/Sem_2_Spring_22/Independent study


In [6]:
from GitHubRepos.TreeMTL.data.nyuv2_dataloader_adashare import NYU_v2
from GitHubRepos.TreeMTL.data.pixel2pixel_loss import NYUCriterions, TaskonomyCriterions
from GitHubRepos.TreeMTL.data.pixel2pixel_metrics import NYUMetrics, TaskonomyMetrics
from GitHubRepos.TreeMTL.main.trainer import Trainer
from GitHubRepos.TreeMTL.main.head import ASPPHeadNode, Classification_Module


In [7]:

tasks = ('segment_semantic','normal','depth_zbuffer')
task_cls_num = {'segment_semantic': 40, 'normal':3, 'depth_zbuffer': 1}

criterionDict = {}
metricDict = {}
clsNum = {}
three_task = ['segment_semantic','normal','depth_zbuffer']

dataroot = "Datasets/nyu_v2"
dataset = NYU_v2(dataroot, 'train', crop_h=321, crop_w=321)
trainDataloader = DataLoader(dataset, 16, shuffle=True)

dataset = NYU_v2(dataroot, 'test', crop_h=321, crop_w=321)
valDataloader = DataLoader(dataset, 16, shuffle=True)

for task in three_task:
    criterionDict[task] = NYUCriterions(task)
    metricDict[task] = NYUMetrics(task)
    clsNum[task] = task_cls_num[task]
print(criterionDict, metricDict, clsNum)

{'segment_semantic': NYUCriterions(
  (cosine_similiarity): CosineSimilarity()
  (l1_loss): L1Loss()
  (l1_loss_sum): L1Loss()
  (cross_entropy): CrossEntropyLoss()
), 'normal': NYUCriterions(
  (cosine_similiarity): CosineSimilarity()
  (l1_loss): L1Loss()
  (l1_loss_sum): L1Loss()
  (cross_entropy): CrossEntropyLoss()
), 'depth_zbuffer': NYUCriterions(
  (cosine_similiarity): CosineSimilarity()
  (l1_loss): L1Loss()
  (l1_loss_sum): L1Loss()
  (cross_entropy): CrossEntropyLoss()
)} {'segment_semantic': <GitHubRepos.TreeMTL.data.pixel2pixel_metrics.NYUMetrics object at 0x7fdb11629fd0>, 'normal': <GitHubRepos.TreeMTL.data.pixel2pixel_metrics.NYUMetrics object at 0x7fdaf9d29a50>, 'depth_zbuffer': <GitHubRepos.TreeMTL.data.pixel2pixel_metrics.NYUMetrics object at 0x7fdaf9d29d50>} {'segment_semantic': 40, 'normal': 3, 'depth_zbuffer': 1}


In [8]:
# data = next(iter(trainDataloader))
# x = data['input']
# y_normal = data['normal']
# print(x.shape)
# print(y_normal.shape)

Importing a pre-trained resnet

In [9]:
class MTLModel(nn.Module):
  def __init__(self, feature_dim=512, clsNum={}, backbone_init=None):
    super(MTLModel, self).__init__()
    self.backbone = backbone_init
    self.heads = nn.ModuleDict()
    # clsNum will contian the name of the task and the number of classes in that task
    for task in clsNum:
      self.heads[task] = ASPPHeadNode(feature_dim, clsNum[task])
  
  def forward(self, x):
    features = self.backbone(x)
    output = {}
    idx = 0
    for task in self.heads:
      output[task] = self.heads[task](features) # why use idx?
      # idx += 1
    return output

In [10]:
model_ft = models.resnet34(pretrained=True)
### strip the last layer
feature_extractor = torch.nn.Sequential(*list(model_ft.children())[:-1])
### check this works
# x = torch.randn([1,3,224,224])
# features = feature_extractor(x) # output now has the features corresponding to input x
# print(features.shape)

In [11]:
model = MTLModel(512, clsNum, feature_extractor)
# print(model)

In [12]:
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, feature_extractor.parameters()), lr=0.001, betas=(0.5, 0.999), weight_decay=0.0001)
optimizer.zero_grad()

In [13]:
loss_lambda = {'segment_semantic': 1, 'normal':1, 'depth_zbuffer': 1}
checkpoint = 'Checkpoints/NYUv2/test/'

In [14]:
# loss = 0
# y = {}
# output = model(x)
# for task in tasks:
# # task = "normal"
# # task = "segment_semantic"
# # task = "depth_zbuffer"
#   tloss = criterionDict[task](output[task], data[task])
#   print("{} loss= {}".format(task, tloss.item())) 
#   loss += loss_lambda[task]*tloss
# loss.backward()
# optimizer.step()

In [16]:
# tb = SummaryWriter()
# data = next(iter(trainDataloader))
# 


In [ ]:
# grid = torchvision.utils.make_grid(data['input'])
# print(grid)

In [17]:
# tb = {} 
tb = SummaryWriter()
# tb.add_image('images', grid)
# tb.add_graph(feature_extractor, data['input'])
# tb.close()

In [18]:
import numpy as np
from sys import exit
import torch

class Trainer():
    def __init__(self, model, tasks, train_dataloader, val_dataloader, criterion_dict, metric_dict, 
                 lr=0.001, decay_lr_freq=4000, decay_lr_rate=0.5,
                 print_iters=50, val_iters=200, save_iters=200):
        print("initializing the Trainer")
        super(Trainer, self).__init__()
        self.model = model
        self.startIter = 0
        self.optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, self.model.parameters()), lr=lr, betas=(0.5, 0.999), weight_decay=0.0001)
        self.scheduler = torch.optim.lr_scheduler.StepLR(self.optimizer, step_size=decay_lr_freq, gamma=decay_lr_rate)
        
        self.tasks = tasks
        
        self.train_dataloader = train_dataloader
        self.train_iter = iter(self.train_dataloader)
        self.val_dataloader = val_dataloader
        self.criterion_dict = criterion_dict
        self.metric_dict = metric_dict
        
        self.loss_list = {}
        self.set_train_loss()
        
        self.print_iters = print_iters
        self.val_iters = val_iters
        self.save_iters = save_iters
        self.tb_data = next(iter(trainDataloader))

    def train(self, iters, loss_lambda, savePath=None, reload=None):
        self.model.train()
        if reload is not None and reload != 'false' and savePath is not None:
            self.load_model(savePath, reload)
        # tb = SummaryWriter()
        # if self.tb_data['input']:
        grid = torchvision.utils.make_grid(self.tb_data['input'])
        tb.add_image('images', grid)
        tb.add_graph(feature_extractor, self.tb_data['input'])
        

        
        for i in range(self.startIter, iters):
            print('i = {}'.format(i))
            self.train_step(loss_lambda)

            if (i+1) % self.print_iters == 0:
                self.print_train_loss(i)
                tb.add_scalar('Loss', self.get_loss(i), i)
                self.set_train_loss()
            if (i+1) % self.val_iters == 0:
                self.validate(i)
            if (i+1) % self.save_iters == 0:
                if savePath is not None:
                    self.save_model(i, savePath)
            # print(self.get_loss(i))
            # tb.add_scalar('Loss', self.get_loss(i), i)
            tb.add_histogram('conv1.weight', feature_extractor[0].weight, i)
            # tb.close()
        # Reset loss list and the data iters
        self.set_train_loss()
        return
    
    def train_step(self, loss_lambda):
        print("starting train step")
        self.model.train()


        try:
            data = next(self.train_iter)
        except StopIteration:
            self.train_iter = iter(self.train_dataloader)
            data = next(self.train_iter)
        
        x = data['input']#.cuda()
        # print("size(x) = ", x.size())
        self.optimizer.zero_grad()
        output = self.model(x)
        # print("output = {}".format(output.size()))
        loss = 0
        for task in self.tasks:
            # print("task = {}".format(task))
            y = data[task]#.cuda()
            if task + '_mask' in data:
                tloss = self.criterion_dict[task](output[task], y, data[task + '_mask'])#.cuda())
            else:
                tloss = self.criterion_dict[task](output[task], y)
                
            self.loss_list[task].append(tloss.item())
            loss += loss_lambda[task] * tloss
            # print("loss = {}".format(loss))
        self.loss_list['total'].append(loss.item())
        print(self.loss_list['total'])
        
        loss.backward()
        self.optimizer.step()
        if self.scheduler is not None:
            self.scheduler.step()
        return
    
    def validate(self, it):
        self.model.eval()
        loss_list = {}
        for task in self.tasks:
            loss_list[task] = []
        
        for i, data in enumerate(self.val_dataloader):
            x = data['input']#.cuda()
            output = self.model(x)

            for task in self.tasks:
                y = data[task]#.cuda()
                if task + '_mask' in data:
                    tloss = self.criterion_dict[task](output[task], y, data[task + '_mask'])#.cuda())
                    self.metric_dict[task](output[task], y, data[task + '_mask'])#.cuda())
                else:
                    tloss = self.criterion_dict[task](output[task], y)
                    self.metric_dict[task](output[task], y)
                loss_list[task].append(tloss.item())

        for task in self.tasks:
            val_results = self.metric_dict[task].val_metrics()
            print('[Iter {} Task {}] Val Loss: {:.4f}'.format((it+1), task[:4], np.mean(loss_list[task])), flush=True)
            print(val_results, flush=True)
        print('======================================================================', flush=True)
        return
    
    # helper functions
    def set_train_loss(self):
        for task in self.tasks:
            self.loss_list[task] = []
        self.loss_list['total'] = []
        return
    
    def load_model(self, savePath, reload):
        model_name = True
        for task in self.tasks:
            if task not in reload:
                model_name = False
                break
        if model_name:
            state = torch.load(savePath + reload)
            self.startIter = state['iter'] + 1
            self.model.load_state_dict(state['state_dict'])
            self.optimizer.load_state_dict(state['optimizer'])
            self.scheduler.load_state_dict(state['scheduler'])
        else:
            print('Cannot load from models trained from different tasks.')
            exit()
        return
    
    def save_model(self, it, savePath):
        state = {'iter': it,
                'state_dict': self.model.state_dict(),
                'layout': self.model.layout,
                'optimizer': self.optimizer.state_dict(),
                'scheduler': self.scheduler.state_dict()}
        if hasattr(self.model, 'branch') and self.model.branch is not None:
            torch.save(state, savePath + '_'.join(self.tasks) + '_b' + str(self.model.branch) + '.model')
        elif hasattr(self.model, 'layout') and self.model.layout is not None:
            torch.save(state, savePath + '_'.join(self.tasks) + '.model')
        return
    
    def print_train_loss(self, it):
        # Function: Print loss for each task
        for task in self.tasks:
            if self.loss_list[task]:
                avg_loss = np.mean(self.loss_list[task])
            else:
                continue
            print('[Iter {} Task {}] Train Loss: {:.4f}'.format((it+1), task[:4], avg_loss), flush=True)
        print('[Iter {} Total] Train Loss: {:.4f}'.format((it+1), np.mean(self.loss_list['total'])), flush=True)
        print('======================================================================', flush=True)
        return
    def get_loss(self, it):
        # for task in self.tasks:
        #     if self.loss_list[task]:
        #         avg_loss = np.mean(self.loss_list[task])
        #     else:
        #         continue
        #     print('[Iter {} Task {}] Train Loss: {:.4f}'.format((it+1), task[:4], avg_loss), flush=True)
        print('[Iter {} Total] Train Loss: {:.4f}'.format((it+1), np.mean(self.loss_list['total'])), flush=True)
        # print('======================================================================', flush=True)
        return np.mean(self.loss_list['total'])

In [26]:
trainer = Trainer(model, three_task, trainDataloader, valDataloader, criterionDict, metricDict, print_iters=1)
trainer.train(100, loss_lambda, checkpoint)

initializing the Trainer
i = 0
starting train step
[5.180189609527588]
[Iter 1 Task segm] Train Loss: 2.5278
[Iter 1 Task norm] Train Loss: 0.2082
[Iter 1 Task dept] Train Loss: 2.4442
[Iter 1 Total] Train Loss: 5.1802
[Iter 1 Total] Train Loss: 5.1802
i = 1
starting train step
[7.208876609802246]
[Iter 2 Task segm] Train Loss: 2.8601
[Iter 2 Task norm] Train Loss: 0.0987
[Iter 2 Task dept] Train Loss: 4.2501
[Iter 2 Total] Train Loss: 7.2089
[Iter 2 Total] Train Loss: 7.2089
i = 2
starting train step
[5.326625823974609]
[Iter 3 Task segm] Train Loss: 2.8814
[Iter 3 Task norm] Train Loss: 0.1618
[Iter 3 Task dept] Train Loss: 2.2834
[Iter 3 Total] Train Loss: 5.3266
[Iter 3 Total] Train Loss: 5.3266
i = 3
starting train step
[4.552000522613525]
[Iter 4 Task segm] Train Loss: 2.8759
[Iter 4 Task norm] Train Loss: 0.1272
[Iter 4 Task dept] Train Loss: 1.5490
[Iter 4 Total] Train Loss: 4.5520
[Iter 4 Total] Train Loss: 4.5520
i = 4
starting train step
[3.0558533668518066]
[Iter 5 Task seg

In [27]:
%tensorboard --logdir runs

Reusing TensorBoard on port 6006 (pid 1285), started 2:19:35 ago. (Use '!kill 1285' to kill it.)

<IPython.core.display.Javascript object>

In [ ]:
# !tensorboard --version

2022-02-27 18:50:49.943161: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2.8.0


In [ ]:
# !tensorboard --logdir=runs

2022-02-27 18:55:52.553699: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.8.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C
